In [0]:
'''
Input: 8x8 mesh simulation
Target: 64x64 mesh simulation
Downsampled skip connection multi scale model with zero padding
'''

In [0]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
import torchvision
import torch.utils.data
import torch.nn.functional as F
import torch.nn.init as init

In [0]:
#Reading training data

train_y = []
train_x = []

for i in range(1, 101, 1):
    temperature = "{:.2f}".format(0.01 * i)
    
    file_path = '8_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '8_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '8_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '8_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
       
    x_input = [u,v,pressure]
    train_x.append(x_input)

    
    file_path = '64_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '64_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '64_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '64_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
    y_output = [u,v,pressure]
    train_y.append(y_output)

In [0]:
#Reading validation data

val_y = []
val_x = []

for i in range(1, 20, 2):
    temperature = 0.05 * i + 0.005
    temperature = "{:.3f}".format(temperature)
    
    file_path = '8_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '8_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '8_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '8_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
    x_input = [u,v,pressure]
    val_x.append(x_input)

    
    file_path = '64_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '64_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '64_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '64_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
    y_output = [u,v,pressure]
    val_y.append(y_output)
    
    
    temperature = 0.05 * i - 0.005
    temperature = "{:.3f}".format(temperature)
    
    file_path = '8_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '8_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '8_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '8_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
   
    x_input = [u,v,pressure]
    val_x.append(x_input)

    
    file_path = '64_u_' + str(temperature) + '.txt'
    u = np.loadtxt(file_path)
    
    file_path = '64_v_' + str(temperature) + '.txt'
    v = np.loadtxt(file_path)
    
    file_path = '64_vel_' + str(temperature) + '.txt'
    vel = np.loadtxt(file_path)
    
    file_path = '64_pressure_' + str(temperature) + '.txt'
    pressure = np.loadtxt(file_path)
    
    y_output = [u,v,pressure]
    val_y.append(y_output)

In [0]:
def show(image):
    plt.imshow(image, vmin=np.amin(image), vmax=np.amax(image), cmap='hsv')
    plt.axis('off')
    plt.show()

In [0]:
from torch.utils.data import Dataset
class dataset(Dataset):
    def __init__(self, train_x, train_y):
        self.x = train_x
        self.y = train_y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [0]:
activate = torch.nn.Tanh()
tanh = torch.nn.Tanh()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.bn0 = nn.BatchNorm2d(num_features = 3)
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 2, stride = 2, padding = 0)
        self.conv2 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 2, stride = 2, padding = 0)
        self.conv3 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 2, stride = 2, padding = 0)
        self.conv4 = nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 4, stride = 2, padding = 1, output_padding=0)
        self.conv5 = nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 4, stride = 2, padding = 1, output_padding=0)
        self.conv6 = nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 4, stride = 2, padding = 1, output_padding=0)
        
        #kernel_size 13
        self.bn7 = nn.BatchNorm2d(num_features = 3)
        self.conv7 = nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 13, stride = 1, padding = 6)
        self.bn8 = nn.BatchNorm2d(num_features = 4)
        self.conv8 = nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 13, stride = 1, padding = 6)
        #kernel_size 9
        self.bn9 = nn.BatchNorm2d(num_features = 3)
        self.conv9  = nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 9, stride = 1, padding = 4)
        self.bn10 = nn.BatchNorm2d(num_features = 4)
        self.conv10 = nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 9, stride = 1, padding = 4)
        #kernel_size 5
        self.bn11 = nn.BatchNorm2d(num_features = 3)
        self.conv11 = nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 5, stride = 1, padding = 2)
        self.bn12 = nn.BatchNorm2d(num_features = 4)
        self.conv12 = nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 5, stride = 1, padding = 2)
        #last conv layer
        self.bn13 = nn.BatchNorm2d(num_features = 24)
        self.conv13 = nn.Conv2d(in_channels = 24, out_channels = 12, kernel_size = 7, stride = 1, padding = 3)
        self.bn14 = nn.BatchNorm2d(num_features = 12)
        self.conv14 = nn.Conv2d(in_channels = 12, out_channels =  3, kernel_size = 5, stride = 1, padding = 2)
        #Final output layer
        self.bn_out = nn.BatchNorm2d(num_features = 3)
        self.out = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 3, stride = 1, padding = 1)
        
    def forward(self, x_64):
        
        x_32 = activate(self.conv1(x_64))
        x_16 = activate(self.conv2(x_32))
        x_8  = activate(self.conv3(x_16))        
        y_16 = activate(self.conv4(x_8))+x_16
        y_32 = activate(self.conv5(y_16))+x_32
        y_64 = activate(self.conv6(y_32))+x_64
        
        z_13 = self.bn7(x_64)
        z_13 = self.conv7(z_13)
        z_13 = activate(z_13)
        z_13 = self.bn8(z_13)
        z_13 = self.conv8(z_13)
        z_13 = activate(z_13)
        
        z_9 = self.bn9(x_64)
        z_9 = self.conv9(z_9)
        z_9 = activate(z_9)
        z_9 = self.bn10(z_9)
        z_9 = self.conv10(z_9)
        z_9 = activate(z_9)
        
        z_5 = self.bn11(x_64)
        z_5 = self.conv11(z_5)
        z_5 = activate(z_5)
        z_5 = self.bn12(z_5)
        z_5 = self.conv12(z_5)
        z_5 = activate(z_5)
        
        z = torch.cat((z_13, z_9, z_5), 1)
        z = self.bn13(z)
        z = self.conv13(z)
        z = activate(z)
        z = self.bn14(z)
        z = self.conv14(z)
        z = activate(z)
        
        z = z+y_64
        z = self.bn_out(z)
        z = self.out(z)
        z = tanh(z)+x_64.resize_(1,3,64,64)
       
        return z

In [0]:
#Unpooling to 64x64 mesh simulation

train_data_x = []
val_data_x = []

for i in range(len(train_x)):
    u = cv2.resize(np.asarray(train_x[i][0]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    v = cv2.resize(np.asarray(train_x[i][1]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    pressure = cv2.resize(np.asarray(train_x[i][2]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    
    train_data_x.append(np.asarray([u,v,pressure]))
    
for i in range(len(val_x)):
    u = cv2.resize(np.asarray(val_x[i][0]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    v = cv2.resize(np.asarray(val_x[i][1]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    pressure = cv2.resize(np.asarray(val_x[i][2]), dsize=(64, 64), interpolation=cv2.INTER_NEAREST)
    
    val_data_x.append(np.asarray([u,v,pressure]))

train_data_x = np.asarray(train_data_x)    
train_y = np.asarray(train_y)    
val_data_x = np.asarray(val_data_x)    
val_y = np.asarray(val_y)    
    
train_data = dataset(train_data_x, train_y)
val_data = dataset(val_data_x, val_y)

In [0]:
def weight_init(m):
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        init.normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [0]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 1, shuffle = False)
test_loader = torch.utils.data.DataLoader(val_data, batch_size = 1, shuffle = False)

In [0]:
criterion = nn.MSELoss()
net = Net()
net.apply(weight_init)
net = net.double()
net = net.cuda()
train_losses = []
val_losses = []

In [0]:
learning_rate = 0.5
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.9)
#optimizer = optim.Adam(net.parameters(), lr = learning_rate, weight_decay = 1e-5)

In [0]:
import matplotlib.pyplot as plt

running_loss = 0.0
num_epochs = 601

for epoch in range(num_epochs):
    running_loss = 0.0
    validation_loss = 0.0
    #print("Epoch: ",epoch+1)
    for i, dat in enumerate((train_loader), 0):
        inputs, labels = dat
        inputs = inputs.resize_(1, 3, 64, 64)
        labels = labels.resize_(1, 3, 64, 64)
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        if(i==9 and epoch%25==0):
            show(outputs[0][0].cpu().detach().numpy())
            show(labels[0][0].cpu().detach().numpy())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    for i, dat in enumerate((test_loader), 0):
        inputs, labels = dat
        inputs = inputs.resize_(1, 3, 64, 64)
        labels = labels.resize_(1, 3, 64, 64)
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        optimizer.step()
        validation_loss += loss.item()
    running_loss/=100.0
    validation_loss/=20.0
    print("Epoch: ",epoch+1," running_loss: ",running_loss*100, "validation_loss: ",validation_loss*100.0)
    train_losses.append(running_loss)
    val_losses.append(validation_loss)

In [0]:
from google.colab import files
torch.save(net.state_dict(),'weights.pth')
files.download('weights.pth')

In [0]:
net.load_state_dict(torch.load("weights.pth"))

In [0]:
train_losses = np.asarray(train_losses)
val_losses = np.asarray(val_losses)
np.savetxt('train.txt', train_losses, delimiter = '\t')
np.savetxt('val.txt', val_losses, delimiter = '\t')
from google.colab import files
files.download('train.txt')
files.download('val.txt')